In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda:0")

In [2]:
train_set = pd.read_excel('./감성대화말뭉치(최종데이터)_Training.xlsx')
validation_set = pd.read_excel('./감성대화말뭉치(최종데이터)_Validation.xlsx')

In [3]:
train_set['감정_소분류'].unique()

array(['노여워하는', '느긋', '걱정스러운', '당혹스러운', '당황', '마비된', '만족스러운', '배신당한',
       '버려진', '부끄러운', '분노', '불안', '비통한', '상처', '성가신', '스트레스 받는', '슬픔',
       '신뢰하는', '신이 난', '실망한', '악의적인', '안달하는', '안도', '억울한', '열등감', '염세적인',
       '외로운', '우울한', '고립된', '좌절한', '후회되는', '혐오스러운', '한심한', '자신하는', '기쁨',
       '툴툴대는', '남의 시선을 의식하는', '회의적인', '죄책감의', '혼란스러운', '초조한', '흥분',
       '충격 받은', '취약한', '편안한', '방어적인', '질투하는', '두려운', '눈물이 나는', '짜증내는',
       '조심스러운', '낙담한', '환멸을 느끼는', '희생된', '감사하는', '구역질 나는', '괴로워하는',
       '가난한, 불우한'], dtype=object)

In [4]:
from sklearn.preprocessing import LabelEncoder


encoder = LabelEncoder()
train_set['감정_소분류'] = encoder.fit_transform(train_set['감정_소분류'])
train_set['감정_소분류'].head()

0    9
1    9
2    9
3    9
4    9
Name: 감정_소분류, dtype: int64

In [5]:
train_set.head(10)

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,9,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,9,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,9,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,9,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,9,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
5,6,청년,여성,"진로,취업,직장",해당없음,분노,9,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.,진로에 대해서 고민하고 계시는군요. 어떤 점이 고민인가요?,직장 상사한테 자주 지적을 받아. 그럴 때마다 이 업무는 나랑 맞지 않는 거 같이 ...,업무가 나와 맞지 않아 시간을 버리는 것 같이 느껴지셨군요.,NaN,NaN
6,7,청년,여성,"진로,취업,직장",해당없음,분노,9,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.,부모님의 노여움에 섭섭하시군요. 이런 상황을 어떻게 해결하면 좋을까요?,부모님께 좀 더 시간을 달라고 해야겠어. 진로 테스트를 받아볼까 생각 중이야.,진로 테스트를 받아보겠다고 부모님께 말씀드릴 생각이군요.,NaN,NaN
7,8,청년,여성,"진로,취업,직장",해당없음,기쁨,11,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,천천히라도 직장을 구해 보려고 하시는군요. 특별한 이유가 있으신가요?,더 좋은 회사가 기다리고 있을지도 모르는데 쉬고 있기가 좀 그래.,좋은 일자리를 놓치고 싶지 않아 끊임없이 노력하시는군요. 하는 일이 잘 풀리시길 바라요.,NaN,NaN
8,9,청년,여성,"진로,취업,직장",해당없음,불안,2,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.,취업에 대해 걱정이 되는군요.,응. 느긋함을 느끼는 내가 좀 싫었는데 걱정을 많이 하는 것보다는 낫다는 생각도 들어.,걱정보다는 느긋한 태도가 낫다고 생각하시는군요.,NaN,NaN
9,10,청년,여성,"진로,취업,직장",해당없음,기쁨,11,요즘 직장생활이 너무 편하고 좋은 것 같아!,직장생활이 편하고 좋으시다니 좋아 보여요. 다니고 계신 회사만의 장점이 있나요?,우리 회사는 복지가 정말 좋아! 내가 곧 출산하게 되는데 육아휴직 기간도 긴 데다가...,예비 엄마들에게는 정말 꿈의 회사인 것 같네요. 마음이 편하시겠어요.,NaN,NaN


In [6]:
new_table = train_set[['감정_소분류','사람문장1','시스템문장1']]
new_table.columns=['label','Q','A']

new_table2 = train_set[['감정_소분류','사람문장2','시스템문장2']]
new_table2.columns=['label','Q','A']

new_table3 = train_set[['감정_소분류','사람문장3','시스템문장3']]
new_table3.columns=['label','Q','A']

In [7]:
new_table.isna().sum()

label    0
Q        0
A        0
dtype: int64

In [8]:
new_table2.isna().sum()

label    0
Q        0
A        0
dtype: int64

In [9]:
new_table3.isna().sum()

label       0
Q        8935
A        8935
dtype: int64

In [10]:
new_table3.dropna(inplace=True)

/tmp/ipykernel_246707/3542675982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_table3.dropna(inplace=True)


In [11]:
chatbottable = pd.concat([new_table,new_table2,new_table3])

In [12]:
chatbottable.head(10)

,label,Q,A
0,9,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,9,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,9,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,9,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,9,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
5,9,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.,진로에 대해서 고민하고 계시는군요. 어떤 점이 고민인가요?
6,9,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.,부모님의 노여움에 섭섭하시군요. 이런 상황을 어떻게 해결하면 좋을까요?
7,11,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,천천히라도 직장을 구해 보려고 하시는군요. 특별한 이유가 있으신가요?
8,2,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.,취업에 대해 걱정이 되는군요.
9,11,요즘 직장생활이 너무 편하고 좋은 것 같아!,직장생활이 편하고 좋으시다니 좋아 보여요. 다니고 계신 회사만의 장점이 있나요?


In [13]:
chatbottable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145954 entries, 0 to 51629
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   145954 non-null  int64 
 1   Q       145954 non-null  object
 2   A       145954 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.5+ MB


In [14]:
chatbottable['label'].value_counts()

label
3     5124
52    5024
2     3475
44    2978
10    2819
38    2816
14    2791
34    2787
9     2779
25    2752
26    2734
27    2719
4     2714
0     2711
21    2704
23    2687
24    2655
48    2635
50    2600
46    2597
18    2579
55    2556
7     2546
20    2542
43    2535
45    2530
22    2507
8     2505
5     2501
42    2498
53    2497
32    2484
41    2477
37    2475
35    2466
47    2466
30    2464
19    2448
13    2447
40    2440
54    2416
31    2395
12    2389
15    2385
57    2370
51    2369
36    2325
17    2294
6     1835
16    1810
33    1757
29    1738
1     1737
28    1674
49    1654
11    1586
39    1586
56    1570
Name: count, dtype: int64

In [ ]:
chatbottable.to_csv("../Question-Emotion_Training/custom_chatbotdataset(Training).csv", index=False, encoding = 'utf-8')

## Validation data 에 대해 똑같이 진행

In [16]:
validation_set['감정_소분류'].unique()

array(['두려운', '마비된', '외로운', '신뢰하는', '남의 시선을 의식하는', '구역질 나는', '낙담한',
       '안달하는', '억울한', '눈물이 나는', '감사하는', '방어적인', '당황', '악의적인', '죄책감의',
       '우울한', '가난한, 불우한', '열등감', '슬픔', '편안한', '느긋', '툴툴대는', '짜증내는', '한심한',
       '후회되는', '분노', '배신당한', '질투하는', '좌절한', '자신하는', '고립된', '충격 받은', '버려진',
       '당혹스러운', '취약한', '스트레스 받는', '실망한', '환멸을 느끼는', '초조한', '회의적인', '불안',
       '부끄러운', '만족스러운', '희생된', '혐오스러운', '혼란스러운', '조심스러운', '안도', '신이 난',
       '상처', '성가신', '괴로워하는', '기쁨', '노여워하는', '염세적인', '걱정스러운', '흥분', '비통한'],
      dtype=object)

In [17]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
validation_set['감정_소분류'] = encoder.fit_transform(validation_set['감정_소분류'])
validation_set['감정_소분류'].head()

0    14
1    14
2    14
3    14
4    14
Name: 감정_소분류, dtype: int64

In [18]:
validation_set.head(10)

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,남성,"진로,취업,직장",해당없음,불안,14,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,실수하시다니 정말 미안한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 될거같아.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.
1,2,청년,남성,"진로,취업,직장",해당없음,불안,14,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,동료 직원에게 도움을 요청하기로 하셨군요.
2,3,청년,남성,"진로,취업,직장",해당없음,불안,14,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.,무섭게 생겼는데도 업무를 보려면 친해져야 할 것 같단말이야.,상사분과 친해질 수 있는 방법은 무엇이 있을까요?,먼저 다가가서 말을 걸어볼게.,직장 상사와 친해지시면 좋겠네요.
3,4,청년,남성,"진로,취업,직장",해당없음,불안,14,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.,첫 직장이어서 잘 적응을 할 수 있을지 모르겠어.,잘 적응 하시려면 무엇을 할 수 있을까요?,직장 동료와 상사들이랑 친하게 지내야겠어.,직장에 잘 적응하시길 바라요.
4,5,청년,남성,"진로,취업,직장",해당없음,불안,14,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,직장 사람들 관계에 대해서 고민이시군요.,내가 낯가림이 심해서 친해질 수 있을지 모르겠어.,직장 사람들과 친해지려면 무슨 방법이 있을까요?,서로 같은 취미를 공유하고 얘기를 나누다 보면 친해질 수 있을 거 같아.,직장 사람들과 좋은 관계를 가지시길 응원해요.
5,6,청년,남성,"진로,취업,직장",해당없음,불안,14,내가 평소에 말이 좀 험한 편인데 무서운 팀장님 앞에서 말실수할까 봐 걱정이야.,팀장님 앞에서 말실수를 할까 봐 두려우신가봐요.,팀장님 앞에서만 있으면 떨려.,말실수 안 하는 법은 무엇이 있나요?,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.
6,7,청년,남성,"진로,취업,직장",해당없음,불안,14,내 직급이 낮아서 그런지 회의 시간에 내 의견을 무시하는 경향이 있어서 의견을 제시...,의견을 무시당해서 의견을 내기가 힘드시군요.,무엇 때문에 그러는지 모르겠어. 하지만 바뀌고 싶어.,의견을 제시하기 위한 방법으로는 무엇이 있나요?,집에서 미리 생각해서 회의 시간에 자신감 있게 말해야겠다.,회의 시간에 의견을 내셔서 원하시는 결과를 얻으시길 바라요.
7,8,청년,남성,"진로,취업,직장",해당없음,불안,14,부장님께 보고서를 올렸는데 혹시 실수한 거 지적당할까 봐 걱정이야.,실수를 하셔서 걱정이시군요.,내가 한 실수 때문에 큰 피해가 갈까 봐 두려워.,큰 피해가 안 일어날 방법은 무엇이 있을까요?,실수한 부분을 제대로 말하고 수정해서 올려야겠어.,좋은 결과가 나오길 바라요
8,9,청년,남성,"진로,취업,직장",해당없음,불안,14,내일 회사 면접을 보러 가는데 내가 틱 증상이 있어서 불안해. 떨어질까 봐 걱정이야.,틱 증상 때문에 걱정이 많으시겠어요.,내가 틱 증상이 있어도 심사위원들에게 잘 보일 수 있을지 모르겠어. 불안하네.,틱 증상이 있으셔도 잘 보일 수 있는 방법은 무엇이 있을까요?,솔직하게 이야기하고 보다 자신감 있게 말해서 난 할 수 있다고 해야겠어.,자신감 있게 임하셔서 좋은 결과 있기를 바라요.
9,10,청년,남성,"진로,취업,직장",해당없음,불안,14,친구랑 같이 백수였는데 내가 이번에 취업을 하는 바람에 그 친구와 멀어질까 봐 걱정이야.,친구분이랑 사이가 멀어질까 봐 걱정이시군요.,난 그 친구랑 안 멀어지고 잘 지내고 싶어.,안 멀어지고 잘 지낼 수 있는 방법은 무엇이 있을까요?,앞에서 회사 얘기는 조금하고 나도 그 친구 취업을 도와줘야겠어.,친구분도 취업하셔서 사이가 안 멀어지시면 좋겠네요.


In [19]:
validation_new_table = validation_set[['감정_소분류','사람문장1','시스템문장1']]
validation_new_table.columns=['label','Q','A']

validation_new_table2 = validation_set[['감정_소분류','사람문장2','시스템문장2']]
validation_new_table2.columns=['label','Q','A']

validation_new_table3 = validation_set[['감정_소분류','사람문장3','시스템문장3']]
validation_new_table3.columns=['label','Q','A']

In [20]:
validation_new_table.isna().sum()

label    0
Q        0
A        0
dtype: int64

In [21]:
validation_new_table2.isna().sum()

label    0
Q        0
A        0
dtype: int64

In [22]:
validation_new_table3.isna().sum()

label       0
Q        1955
A        1955
dtype: int64

In [23]:
validation_new_table3.dropna(inplace=True)

/tmp/ipykernel_246707/76753375.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_new_table3.dropna(inplace=True)


In [24]:
validation_chatbottable = pd.concat([validation_new_table,validation_new_table2,validation_new_table3])

In [25]:
validation_chatbottable.head(10)

,label,Q,A
0,14,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,실수하시다니 정말 미안한 마음이 크겠어요.
1,14,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.
2,14,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.
3,14,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.
4,14,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,직장 사람들 관계에 대해서 고민이시군요.
5,14,내가 평소에 말이 좀 험한 편인데 무서운 팀장님 앞에서 말실수할까 봐 걱정이야.,팀장님 앞에서 말실수를 할까 봐 두려우신가봐요.
6,14,내 직급이 낮아서 그런지 회의 시간에 내 의견을 무시하는 경향이 있어서 의견을 제시...,의견을 무시당해서 의견을 내기가 힘드시군요.
7,14,부장님께 보고서를 올렸는데 혹시 실수한 거 지적당할까 봐 걱정이야.,실수를 하셔서 걱정이시군요.
8,14,내일 회사 면접을 보러 가는데 내가 틱 증상이 있어서 불안해. 떨어질까 봐 걱정이야.,틱 증상 때문에 걱정이 많으시겠어요.
9,14,친구랑 같이 백수였는데 내가 이번에 취업을 하는 바람에 그 친구와 멀어질까 봐 걱정이야.,친구분이랑 사이가 멀어질까 봐 걱정이시군요.


In [26]:
validation_chatbottable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17968 entries, 0 to 6580
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   17968 non-null  int64 
 1   Q       17968 non-null  object
 2   A       17968 non-null  object
dtypes: int64(1), object(2)
memory usage: 561.5+ KB


In [27]:
validation_chatbottable['label'].value_counts()

label
52    571
3     545
21    503
49    496
28    434
30    423
26    407
12    373
41    370
14    370
46    369
31    366
22    363
13    358
1     355
5     354
32    350
42    349
48    349
37    346
50    344
10    338
44    336
16    331
9     326
33    326
53    324
39    311
20    305
55    303
38    299
43    296
29    288
2     285
45    284
8     279
40    279
57    276
11    273
7     264
34    262
4     258
6     255
19    253
17    249
24    242
36    238
27    236
54    232
15    228
18    221
47    207
25    207
51    194
0     176
56    146
23    131
35    115
Name: count, dtype: int64

In [28]:
validation_chatbottable.to_csv("../Question-Emotion_Training/custom_chatbotdataset(Validation).csv", index=False, encoding = 'utf-8')